# <center>Đồ án 3: Linear Regression</center>

# Thông tin sinh viên

- Họ và tên: Lâm Thanh Ngọc
- MSSV: 21127118
- Lớp: 21CLC02

# Import

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
# Import thêm dữ thư viện nếu cần

# Đọc dữ liệu

In [2]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện) -> X_1a
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

# Cài đặt hàm

In [3]:
# Cài đặt các hàm cần thiết ở đây
class OLSLinearRegression:
    def fit(self, X, y):
        X_pinv = np.linalg.inv(X.T @ X) @ X.T    # np.linalg.pinv(X)
        self.w = X_pinv @ y

        return self

    def get_params(self):
        return self.w


    def predict(self, X):
        return np.sum(self.w.ravel() * X, axis=1)

In [4]:
def mae(y, y_hat):
    return np.mean(np.abs(y.ravel() - y_hat.ravel()))

# Yêu cầu 1a: Sử dụng toàn bộ 11 đặc trưng đầu tiên `Gender`, `10percentage`, `12percentage`, `CollegeTier`, `Degree`, `collegeGPA`, `CollegeCityTier`, `English`, `Logical`, `Quant`, `Domain` (2 điểm) 

In [5]:
# Phần code cho yêu cầu 1a
X_1a_train = train.iloc[:, :11]
X_1a_test = test.iloc[:, :11]

lr_1a = OLSLinearRegression().fit(X_1a_train, y_train)
lr_1a.get_params()
Y_1a_test = lr_1a.predict(X_1a_test)

In [6]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra
mae(y_test, Y_1a_test)

104863.77754033149

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = \text{Gender} * (-22756.513) + \text{10percentage} * 804.503 + \text{12percentage} * 1294.655$$
$$ + \text{CollegeTier} * (-91781.898) + \text{Degree} * 23182.389 + \text{collegeGPA} * 1437.549 + \text{CollegeCityTier}$$
$$ * (-8570.662) + \text{English} * 147.858 + \text{Logical} * 152.888 + \text{Quant} * 117.222 + \text{Domain} * 34552.286$$

# Yêu cầu 1b: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng tính cách với các đặc trưng tính cách gồm `conscientiousness`, `agreeableness`, `extraversion`, `nueroticism`, `openess_to_experience`, tìm mô hình cho kết quả tốt nhất (1 điểm) 

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [7]:
# Phần code cho yêu cầu 1b

X_1b_cv_train = train.iloc[:, -6:-1] # Lấy ra các đặc trưng tính cách của tập train (5 đặc trưng)
X_1b_cv_test = test.iloc[:, -6:-1] # Lấy ra các đặc trưng tính cách của tập test (5 đặc trưng)

mean_mae = []

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for i in X_1b_cv_train.keys():
    mae_list = [] # Danh sách lưu các độ lỗi MAE của mô hình với các đặc trưng
    
    for train, test in kfold.split(X_1b_cv_train):
        # Lấy ra đặc trưng thứ i của tập train
        X_cv_train, X_cv_test = X_1b_cv_train.loc[train, [i]], X_1b_cv_train.loc[test, [i]]
        y_cv_train, y_cv_test = y_train[train], y_train[test]

        lr_cv_1b = OLSLinearRegression().fit(X_cv_train, y_cv_train)
        y_cv_1b = lr_cv_1b.predict(X_cv_test)
        
        mae_list.append(mae(y_cv_test, y_cv_1b)) # Tính độ lỗi MAE của mô hình với đặc trưng thứ i
        
    mean_mae.append(np.mean(mae_list)) # Tính độ lỗi MAE trung bình của mô hình với đặc trưng thứ i

# Tìm ra đặc trưng tốt nhất
best_feature = X_1b_cv_train.keys()[mean_mae.index(min(mean_mae))]
print("Best feature: ", best_feature)

# In ra các kết quả cross-validation như yêu cầu
print("MAE: ", mean_mae)

Best feature:  nueroticism
MAE:  [306309.20177532476, 300912.6776780873, 307030.1026896982, 299590.04982342874, 302957.69185422716]


<center>

| STT | Mô hình với 1 đặc trưng | MAE  |
|:---:|:-----------------------:|:----:|
|  1  | conscientiousness       |306309.202|
|  2  | agreeableness           |300912.678|
|  3  | extraversion            |307030.103|
|  4  | neuroticism             |299590.050|
|  5  | openness_to_experience  |302957.692|

</center>

In [8]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện) -> X_1a
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

In [9]:
# Huấn luyện lại mô hình best_personality_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
best_personality_feature_model = train.loc[:, [best_feature]]
X_1b_test = test.loc[:, [best_feature]]

lr_1b = OLSLinearRegression().fit(best_personality_feature_model, y_train)
lr_1b.get_params()
print(lr_1b.get_params())
y_1b = lr_1b.predict(X_1b_test)

0   -56546.303753
dtype: float64


In [10]:
# Gọi hàm MAE (tự cài đặt hoặc từ thư viện) trên tập kiểm tra với mô hình best_personality_feature_model
mae(y_test, y_1b)

291019.693226953

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = (-56546.304) * nueroticism$$

# Yêu cầu 1c: Xây dựng mô hình sử dụng duy nhất 1 đặc trưng `English`, `Logical`, `Quant`, tìm mô hình cho kết quả tốt nhất (1 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ đặc trưng

In [11]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện) -> X_1a
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

In [12]:
# Phần code cho yêu cầu 1c

X_1c_cv_train = train.iloc[:, 7:10] # Lấy ra các đặc trưng tính cách của tập train (5 đặc trưng)
X_1c_cv_test = test.iloc[:, 7:10] # Lấy ra các đặc trưng tính cách của tập test (5 đặc trưng)

mean_mae = []

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for i in X_1c_cv_train.keys():
    mae_list = [] # Danh sách lưu các độ lỗi MAE của mô hình với các đặc trưng
    
    for train, test in kfold.split(X_1c_cv_train):
        # Lấy ra đặc trưng thứ i của tập train
        X_cv_train, X_cv_test = X_1c_cv_train.loc[train, [i]], X_1c_cv_train.loc[test, [i]]
        y_cv_train, y_cv_test = y_train[train], y_train[test]

        lr_cv_1c = OLSLinearRegression().fit(X_cv_train, y_cv_train)
        y_cv_1c = lr_cv_1c.predict(X_cv_test)
        
        mae_list.append(mae(y_cv_test, y_cv_1c))
        
    mean_mae.append(np.mean(mae_list)) # Tính độ lỗi MAE của mô hình với đặc trưng thứ i

# Tìm ra đặc trưng tốt nhất
best_skill = X_1c_cv_train.keys()[mean_mae.index(min(mean_mae))]
print("Best skill: ", best_skill)

# In ra các kết quả cross-validation như yêu cầu
print("MAE: ", mean_mae)

Best skill:  Quant
MAE:  [121925.88431534174, 120274.77773746487, 118124.52445576759]


<center>

| STT | Mô hình với 1 đặc trưng | MAE  |
|:---:|:-----------------------:|:----:|
|  1  | English			        |121925.884|
|  2  | Logical		            |120274.778|
|  3  | Quant		            |118124.524|

</center>

In [13]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện) -> X_1a
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

# Sinh viên có thể sử dụng các khác nếu cần

In [14]:
# Huấn luyện lại mô hình best_skill_feature_model với đặc trưng tốt nhất trên toàn bộ tập huấn luyện
best_skill_feature_model = train.loc[:, [best_skill]]
X_1c_test = test.loc[:, [best_feature]]

lr_1c = OLSLinearRegression().fit(best_skill_feature_model, y_train)
lr_1c.get_params()
#print(lr_1c.get_params())
y_1c = lr_1c.predict(X_1c_test)

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = 585.895 * Quant$$

# Yêu cầu 1d: Sinh viên tự xây dựng mô hình, tìm mô hình cho kết quả tốt nhất (3 điểm)

Lưu ý: khi sử dụng cross-validation, sinh viên cần xáo trộn dữ liệu 1 lần duy nhất và thực hiện trên toàn bộ $m$ mô hình mà sinh viên thiết kế

## Tìm mô hình

In [15]:
# Trình bày các phần tìm ra mô hình

## Thử nghiệm, so sánh các mô hình

In [16]:
# Phần code cho yêu cầu 1d
# Tìm ra mô hình tốt nhất (tự thiết kế bởi sinh viên)
# In ra các kết quả cross-validation như yêu cầu

    Mô hình 1

In [17]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)


In [18]:
# Phần code cho yêu cầu 1d

X_1d_cv_train_1 = X_train
X_1d_cv_test_1 = X_test

mae_1 = []

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

for train, test in kfold.split(X_1d_cv_train_1):
    # Lấy ra đặc trưng thứ i của tập train
    X_cv_train_1, X_cv_test_1 = X_1d_cv_train_1.loc[train], X_1d_cv_train_1.loc[test]
    y_cv_train_1, y_cv_test_1 = y_train[train], y_train[test]

    lr_cv_1d_1 = OLSLinearRegression().fit(X_cv_train_1, y_cv_train_1)
    y_cv_1d_1 = lr_cv_1d_1.predict(X_cv_test_1)

    mae_1.append(mae(y_cv_test_1, y_cv_1d_1))

mean_mae_1 = np.mean(mae_1)


    Mô hình 2

In [19]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)


In [20]:
X_1d_cv_train_2 = train.iloc[:, 3:-1]
X_1d_cv_test_2 = test.iloc[:, 3:-1]

mae_2 = []

# for i in X_1d_cv_train_2.keys():
#     mae_list = [] # Danh sách lưu các độ lỗi MAE của mô hình với các đặc trưng
for train2, test2 in kfold.split(X_1d_cv_train_2):
    # Lấy ra đặc trưng thứ i của tập train
    X_cv_train_2, X_cv_test_2 = X_1d_cv_train_2.loc[train2], X_1d_cv_train_2.loc[test2]
    y_cv_train_2, y_cv_test_2 = y_train[train2], y_train[test2]

    lr_cv_1d_2 = OLSLinearRegression().fit(X_cv_train_2, y_cv_train_2)
    y_cv_1d_2 = lr_cv_1d_2.predict(X_cv_test_2)

    mae_2.append(mae(y_cv_test_2, y_cv_1d_2))
#mae_2.append(np.mean(mae_list)) # Tính độ lỗi MAE của mô hình với đặc trưng thứ i
    
mean_mae_2 = np.mean(mae_2)

    Mô hình 3

In [21]:
# Đọc dữ liệu bằng pandas
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Lấy các đặc trưng X và giá trị mục tiêu y cho các tập huấn luyện (train) và kiểm tra (test)
X_train = train.iloc[:, :-1]    # Dataframe (chứa các đặc trưng huấn luyện)
y_train = train.iloc[:, -1]     # Series    (chứa 1 giá trị mục tiêu kiểm tra)

X_test = test.iloc[:, :-1]      # Dataframe (chứa các đặc trưng kiểm tra)
y_test = test.iloc[:, -1]       # Series    (chứa 1 giá trị mục tiêu kiểm tra)

In [22]:
X_1d_cv_train_3 = pd.concat([X_1a_train, X_1b_cv_train], axis=1)
X_1d_cv_test_3 = pd.concat([X_1a_test, X_1b_cv_test], axis=1)

mae_3 = []

# for i in X_1d_cv_train_3.keys():
#     mae_list = [] # Danh sách lưu các độ lỗi MAE của mô hình với các đặc trưng
for train3, test3 in kfold.split(X_1d_cv_train_3):
    # Lấy ra đặc trưng thứ i của tập train
    X_cv_train_3, X_cv_test_3 = X_1d_cv_train_3.loc[train3], X_1d_cv_train_3.loc[test3]
    y_cv_train_3, y_cv_test_3 = y_train[train3], y_train[test3]

    lr_cv_1d_3 = OLSLinearRegression().fit(X_cv_train_3, y_cv_train_3)
    y_cv_1d_3 = lr_cv_1d_3.predict(X_cv_test_3)

    mae_3.append(mae(y_cv_test_3, y_cv_1d_3))
#mae_3.append(np.mean(mae_list)) # Tính độ lỗi MAE của mô hình với đặc trưng thứ i
    
mean_mae_3 = np.mean(mae_3)

    Mô hình tốt nhất

In [23]:
# Tìm ra mô hình tốt nhất

best_model = [mean_mae_1, mean_mae_2, mean_mae_3].index(min([mean_mae_1, mean_mae_2, mean_mae_3]))
print("Best model: ", best_model)

# In ra các kết quả cross-validation như yêu cầu
print("MAE: ", [mean_mae_1, mean_mae_2, mean_mae_3])

Best model:  0
MAE:  [110420.41377587737, 111724.32684927108, 113088.91774985948]


<center>

| STT |                     Mô hình                       |     MAE    |
|:---:|:-------------------------------------------------:|:----------:|
|  1  | Sử dụng toàn bộ đặc trưng                         | 110420.414 |
|  2  | Bỏ 3 đặc tính Gender, 10percentage, 12percentage  | 111724.327 |
|  3  | Gộp các đặc trưng câu 1a và 1b                    | 113088.918 |

</center>

In [24]:
X_1d_train = X_1d_cv_train_1
X_1d_test = X_1d_cv_test_1

lr_1d = OLSLinearRegression().fit(X_1d_train, y_train)
lr_1d.get_params()
print(lr_1d.get_params())
y_1d = lr_1d.predict(X_1d_test)

0    -23874.541727
1       898.575621
2      1203.496112
3    -83592.387591
4     11515.430757
5      1626.518605
6     -5717.733852
7       153.434567
8       120.511333
9       102.580853
10    27939.639602
11       76.730246
12      -47.746793
13     -177.387649
14       33.932559
15     -151.471153
16      -64.197706
17      145.894996
18   -19814.830268
19    15503.266941
20     4908.582006
21   -10661.029100
22    -5815.021280
dtype: float64


In [25]:
mae(y_test, y_1d)

101872.2105661925

Công thức hồi quy (phần trọng số làm tròn đến 3 chữ số thập phân, ví dụ 0.012345 $\to$ 0.012)

$$\text{Salary} = -23874.542 * Gender + 898.576 * 10percentage + 1203.496 * 12percentage -83592.388$$ 
$$ * CollegeTier + 11515.431 * Degree + 1626.519 * CollegeGPA - 5717.734 * CollegeCityTier$$
$$ + 153.435 * English + 120.511 * Logical + 102.581 * Quant + 27939.640 * Domain + 76.730 * $$ 
$$ ComputerProgramming - 47.747 * ElectronicsAndSemicon - 177.387 * ComputerScience + 33.932 * $$ 
$$ MechanicalEngg - 151.471 * ElectricalEngg - 64.198 * TelecomEngg + 145.894 * CivilEngg $$ 
$$ - 19814.830 * conscientiousness + 15503.266 * agreeableness + 4908.582 * extraversion$$
$$ - 10661.029 * nueroticism - 5815.021 * openess\_to\_experience$$